# Download MassBank Dataset

In [1]:
! git clone https://github.com/MassBank/MassBank-data.git 
! wget https://github.com/TakashiKusachi/icml18-jtnn/raw/master/data/zinc/all.txt

fatal: destination path 'MassBank-data' already exists and is not an empty directory.
--2020-01-21 21:22:21--  https://github.com/TakashiKusachi/icml18-jtnn/raw/master/data/zinc/all.txt
cache.ccs.kogakuin.ac.jp (cache.ccs.kogakuin.ac.jp) をDNSに問いあわせています... 133.80.8.17, 2001:df0:456:603::17
cache.ccs.kogakuin.ac.jp (cache.ccs.kogakuin.ac.jp)|133.80.8.17|:8080 に接続しています... 接続しました。
Proxy による接続要求を送信しました、応答を待っています... 302 Found
場所: https://raw.githubusercontent.com/TakashiKusachi/icml18-jtnn/master/data/zinc/all.txt [続く]
--2020-01-21 21:22:22--  https://raw.githubusercontent.com/TakashiKusachi/icml18-jtnn/master/data/zinc/all.txt
cache.ccs.kogakuin.ac.jp (cache.ccs.kogakuin.ac.jp)|133.80.8.17|:8080 に接続しています... 接続しました。
Proxy による接続要求を送信しました、応答を待っています... 200 OK
長さ: 11303042 (11M) [text/plain]
`all.txt.1' に保存中

all.txt.1           100%[===================>]  10.78M  43.6MB/s    時間 0.2s  

2020-01-21 21:22:22 (43.6 MB/s) - `all.txt.1' へ保存完了 [11303042/11303042]



In [5]:
import os
from glob import glob
from pathlib import Path
from multiprocessing import Pool
from rdkit import Chem
import pickle
import numpy as np


In [6]:
file_list = list(Path("./MassBank-data").glob("*/*.txt"))
print("number of file list: {}",format(len(file_list)))

number of file list: {} 76694


In [7]:
NUMBET_OF_MP_PROCESSES = None # Number of processes use to parse and expand data.

AUTHORS = "AUTHORS"
AC_INSTRUMENT = "AC$INSTRUMENT"
AC_INSTRUMENT_TYPE = "AC$INSTRUMENT_TYPE"
CH_NAME = "CH$NAME"
CH_SMILES = "CH$SMILES"
AC_MASS_SPECTROETRY = "AC$MASS_SPECTROMETRY"
MS_TYPE = "MS_TYPE"
ION_MODE = "ION_MODE"
IONIZATION_ENERGY = "IONIZATION_ENERGY"

def analyze(args):
    num,path = args
    #param init 
    auth = None
    inst = None
    inst_type = None
    name = None
    smiles = None
    ms_type = None
    ion_mode = None
    ret = {}
    with open(path) as f:
        temp = f.read().split("\n")
        for num,line in enumerate(temp):
            row = line.split(": ",1)
            if row[0] == AUTHORS:
                ret["authors"]= row[1]
            elif row[0] == AC_INSTRUMENT:
                ret["instrument"] = row[1]
            elif row[0] == AC_INSTRUMENT_TYPE:
                ret["instrument_type"] = row[1]
            elif row[0] == CH_NAME:
                ret["name"] = row[1]
            elif row[0] == CH_SMILES:
                mol = Chem.MolFromSmiles(row[1])
                if mol is None:
                    ret["smiles"] = row[1]
                else:
                    ret["smiles"] = Chem.MolToSmiles(mol)
            elif row[0] == AC_MASS_SPECTROETRY:
                label,_type = row[1].split(" ",1)
                if label == MS_TYPE:
                    ret["ms_type"] = _type
                elif label == ION_MODE:
                    ret["ion_mode"] = _type
                elif label == IONIZATION_ENERGY:
                    ret["ionization_energy"] = _type
            elif row[0] == "PK$PEAK":
                peak_start = num+1
        peak_x,peak_y = zip(*[one.split(" ")[2:4] for one in temp[peak_start:-1] if one != "//"])
        ret["peak_x"] = np.array(peak_x,dtype=np.float32)
        ret["peak_y"] = np.array(peak_y,dtype=np.float32)
    ret["path"] = path
    return ret

with Pool(processes=NUMBET_OF_MP_PROCESSES) as p:
    ret = p.map(analyze,enumerate(file_list))

print("number of expand data:",len(ret))

with open("massbank.pkl","wb") as f:
    pickle.dump(ret,f,protocol=2)

def is_select(one):
    return one["smiles"]!="N/A"
    
with open("massbank_smiles_list.txt","w") as f:
    f.write("\n".join([one["smiles"] for one in ret if is_select(one)]))

RDKit ERROR: [21:23:14] SMILES Parse Error: syntax error while parsing: N/A
RDKit ERROR: [21:23:14] SMILES Parse Error: Failed parsing SMILES 'N/A' for input: 'N/A'
RDKit ERROR: [21:23:14] SMILES Parse Error: syntax error while parsing: N/A
RDKit ERROR: [21:23:14] SMILES Parse Error: syntax error while parsing: N/A
RDKit ERROR: [21:23:14] SMILES Parse Error: Failed parsing SMILES 'N/A' for input: 'N/A'
RDKit ERROR: [21:23:14] SMILES Parse Error: syntax error while parsing: N/A
RDKit ERROR: [21:23:14] SMILES Parse Error: Failed parsing SMILES 'N/A' for input: 'N/A'
RDKit ERROR: [21:23:14] SMILES Parse Error: syntax error while parsing: N/A
RDKit ERROR: [21:23:14] SMILES Parse Error: syntax error while parsing: N/A
RDKit ERROR: [21:23:14] SMILES Parse Error: Failed parsing SMILES 'N/A' for input: 'N/A'
RDKit ERROR: [21:23:14] SMILES Parse Error: syntax error while parsing: N/A
RDKit ERROR: [21:23:14] SMILES Parse Error: Failed parsing SMILES 'N/A' for input: 'N/A'
RDKit ERROR: [21:23:14]

number of expand data: 76694


In [8]:

zinc_path = "./all.txt"
massbank_path = "./massbank_smiles_list.txt"

print("get zinc list")
with open(zinc_path) as f:
    zinc_mols = [one.strip() for one in f.readlines() if one != "\n"]
print("Number of zinc dataset: %d" % (len(zinc_mols)))
    
print("get massbank list")
with open(massbank_path) as f:
    massbank_mols = [one.strip() for one in f.readlines() if one != "\n"]
print("Number of massbank dataset: %d" % (len(massbank_mols)))

smiles_list = []

def silialize(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return Chem.MolToSmiles(mol)
    #return smiles
    
def callback(smiles):
    global smiles_list
    for one in smiles:
        if one is not None:
            smiles_list.append(one)
        else:
            print("reject")
    

with Pool() as p:
    r = p.map_async(silialize, zinc_mols + massbank_mols, callback=callback)
    while(not r.ready()):
        print("Now Processing")
        r.wait(timeout=60)
    if r.successful() == False:
        print("Processing Error")
        print(r.get())


print("Check Conflict")
result = list(set(smiles_list))
print("Number of Conflicted :%d " %(len(smiles_list)-len(result)))

del zinc_mols,massbank_mols,p

get zinc list
Number of zinc dataset: 249456
get massbank list
Number of massbank dataset: 75706
Now Processing


RDKit ERROR: [21:23:55] Can't kekulize mol.  Unkekulized atoms: 2 4 6
RDKit ERROR: 


reject
Check Conflict
Number of Conflicted :60023 


In [9]:
with open("all_smiles.txt","w") as f:
    f.write("\n".join(result))

In [10]:
del result